In [1]:
import os
import sys
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import Phylo
import numpy as np
import pandas as pd
import seaborn as sns
import pysam
import math
mpl.rcParams['pdf.fonttype'] = 42
%matplotlib inline

In [2]:
def checkSlash(directory):
    if directory[-1] != '/':
        directory = directory + '/'
    return directory

In [7]:
unmappedDir = '12_bwa_unmapped/'
readDir = '13_bwa_horizontal/'
prokkaDir = '09_prokka_biopython/'
prokkaDir = checkSlash(prokkaDir)
annoDir = '14_bwa_annotations'
annoDir = checkSlash(annoDir)
if not os.path.exists(annoDir):
    os.makedirs(annoDir)

In [4]:
#Returns pandas dataframe of variable read mappings for a single sample of source reads
#Input = mappedID: source reads for mapping are from this sample
#        unmapped: directory of unmapped results
#Columns = samples in subgroup, plus 'sequence_1' and 'sequence_2' for the fwd & rev sequences of the read
#Rows = Read IDs
#Data = one if even one read mapped, zero if both paired ends are unmapped
def binaryMapping(mappedID, unmapped):
    unmapped = checkSlash(unmapped)
    unmappedDict = {}
    unmappedFwdSeqs = {}
    unmappedRevSeqs = {}
    for f in os.listdir(unmapped):
        if 'mapped_' + mappedID in f:
            contig = f.split('_')[1]
            inFile = open(unmapped + f, 'r')
            for line in inFile:
                line = line.strip().split('\t')
                readID = line[0]
                flag = line[1]
                seq = line[9]
                if flag == '77':
                    unmappedFwdSeqs[readID] = seq
                elif flag == '141':
                    unmappedRevSeqs[readID] = seq
                if contig not in unmappedDict:
                    unmappedDict[contig] = {}
                if readID not in unmappedDict[contig]:
                    unmappedDict[contig][readID] = 1
                else:
                    unmappedDict[contig][readID] = 2
            inFile.close()
    df = pd.DataFrame.from_dict(unmappedDict)
    df = df.replace(1, np.nan)
    df = df.replace(2, 0)               #0 if both paired-ends unmapped
    df = df.fillna(value=1)             #1 if even one is mapped
    ####
    #Remove reads that are unmapped (e.g. zeros) in all samples
    df = df.loc[(df != 0).any(axis = 1)]
    #Remove reads that are mapped (e.g. ones) in all samples
    df = df.loc[(df.sum(axis=1) != len(df.columns))]
    df['sequence_1'] = ''
    df['sequence_2'] = ''
    for i in list(df.index):
        df['sequence_1'][i] = unmappedFwdSeqs[i]
        df['sequence_2'][i] = unmappedRevSeqs[i]
    return df

In [5]:
#Import dataframe with reads as indexes and samples as columns.
#1's indicate that at least one PE read mapped, 0's mean both paired ends were unmapped
mappedSamp = 'D17-102065'
subgroup = 'subA/'
subgroup = checkSlash(subgroup)
df = binaryMapping(mappedSamp, unmappedDir + subgroup)
#exportBinaryMapping(df, readDir + subgroup + mappedSamp + '_horizontal.fa')

/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/tools/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#reads: list of reads to annotate
#alignmentFile: alignment to contigs
#prokkaFile: prokka-annotated contigs
def importAlignment(reads, alignmentFile):
    annoDf = pd.DataFrame(index = reads, columns = ['ref_id_1', 'ref_pos_1', 'ref_id_2', 'ref_pos_2'])
    sam = pysam.AlignmentFile(alignmentFile, "r")
    ct = 0
    for r in sam:
        if r.qname in reads:
            if r.flag & 4:
                continue
            refName = sam.get_reference_name(r.tid)
            if r.flag & 64:
                annoDf['ref_id_1'][r.qname] = refName
                annoDf['ref_pos_1'][r.qname] = r.pos
            else:
                annoDf['ref_id_2'][r.qname] = refName
                annoDf['ref_pos_2'][r.qname] = r.pos
    return annoDf

def getAnnotation(prokkaFile, fwdNode, fwdPos, revNode, revPos):
    if fwdNode != revNode:
        print('MAPPING ERROR: ' + fwdNode + '\t' + revNode)
    if (math.fabs(fwdPos - revPos)) > 1000:
        print('MAPPING ERROR: fwd & rev reads > 1000 bp apart')
    records = list(SeqIO.parse(prokkaFile, 'genbank'))
    passedMapping = False
    for r in records:
        if r.id == fwdNode:
            prevFeature = np.nan
            for i, f in enumerate(r.features):
                if f.type == 'source':
                    continue
                start = str(f.location).split(':')[0].replace('[', '')
                end = str(f.location).split(':')[1].split(']')[0]
                strand = str(f.location).split('(')[1].replace(')', '')
                #Look for a coding feature
                if (fwdPos > int(start)) and (fwdPos < int(end)):
                    if (revPos > int(start)) and (revPos < int(end)):
                        pass
                    else:
                        print('WARNING: fwd & rev reads map to different features')
                    return {'coding': f}
                #Look for intergenic
                if (fwdPos < int(start)) and (fwdPos < int(end)):
                    if isinstance(prevFeature, float):
                        prevStrand = '+'
                    else:
                        prevStrand = str(prevFeature.location).split('(')[1].replace(')', '')
                    if (prevStrand == '+') and (strand == '+'):
                        return {'downstream1': prevFeature, 'upstream1': f}
                    elif (prevStrand == '-') and (strand == '-'):
                        return {'downstream1': f, 'upstream1': prevFeature}
                    elif (prevStrand == '+') and (strand == '-'):
                        return {'upstream1': prevFeature, 'upstream2': f}
                    else:
                        return {'downstream1': prevFeature, 'downstream2': f}
                #store prev
                prevFeature = f

In [135]:
mappedID = 'D17-102065'
contigID = 'D17-102051'
prokkaFile = prokkaDir + contigID + '.gb'
annoDf = importAlignment(list(df.index), '11_bwa/subA/contig_' + contigID + '_mapped_' + mappedID + '.sam')
featureDf = pd.DataFrame(index = list(annoDf.index), columns = ['coding', 'up1', 'up2',
                                                                'down1', 'down2'])
for i in list(annoDf.index):
    if isinstance(annoDf['ref_id_1'][i], str):
        featureDict = getAnnotation(prokkaFile, annoDf['ref_id_1'][i], annoDf['ref_pos_1'][i], 
                      annoDf['ref_id_2'][i], annoDf['ref_pos_2'][i])
        if isinstance(featureDict, dict): #protect from null values, no annotation
            if 'coding' in featureDict:
                quals = featureDict['coding'].qualifiers
                if 'gene' in quals:
                    featureDf['coding'][i] = quals['gene'][0]
                elif 'product' in quals:
                    featureDf['coding'][i] = quals['product'][0]
            featureList.append(featureDict)
featureDf.to_csv(annoDir + mappedID + '_' + contigID + '_test.txt', sep='\t')
featureDf

,coding,up1,up2,down1,down2
NB501288_148_H2MVCBGX2:1:11105:21972:18990#GGACTCCTAGGCTTAG,hypothetical protein,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:11112:11049:4981#GGACTCCTAGGCTTAG,hypothetical protein,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:11204:18943:15036#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:11207:11840:16657#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:11208:23068:10313#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12101:12780:5384#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12101:14113:4828#GGACTCCTAGGCTTAG,hypothetical protein,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12102:18170:10915#GGACTCCTAGGCTTAG,hypothetical protein,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12102:23008:8043#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12106:23839:17896#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN,NaN


In [133]:
annoDf

,ref_id_1,ref_pos_1,ref_id_2,ref_pos_2
NB501288_148_H2MVCBGX2:1:11105:21972:18990#GGACTCCTAGGCTTAG,NODE_1_length_337279_cov_29.9505,91542,NODE_1_length_337279_cov_29.9505,91744
NB501288_148_H2MVCBGX2:1:11112:11049:4981#GGACTCCTAGGCTTAG,NODE_1_length_337279_cov_29.9505,90991,NODE_1_length_337279_cov_29.9505,91172
NB501288_148_H2MVCBGX2:1:11204:18943:15036#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:11207:11840:16657#GGACTCCTAGGCTTAG,NODE_1_length_337279_cov_29.9505,91436,NODE_1_length_337279_cov_29.9505,91265
NB501288_148_H2MVCBGX2:1:11208:23068:10313#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12101:12780:5384#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12101:14113:4828#GGACTCCTAGGCTTAG,NODE_1_length_337279_cov_29.9505,91510,NODE_1_length_337279_cov_29.9505,91348
NB501288_148_H2MVCBGX2:1:12102:18170:10915#GGACTCCTAGGCTTAG,NODE_1_length_337279_cov_29.9505,91161,NODE_1_length_337279_cov_29.9505,91113
NB501288_148_H2MVCBGX2:1:12102:23008:8043#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN
NB501288_148_H2MVCBGX2:1:12106:23839:17896#GGACTCCTAGGCTTAG,NaN,NaN,NaN,NaN


# Discarded code

In [ ]:
annoDf['coding'] = np.nan
annoDf['bp_up'] = np.nan
annoDf['upstream'] = np.nan
annoDf['bp_down'] = np.nan
annoDf['downstream'] = np.nan
for s in df:
    if (s != 'sequence_1') and (s != 'sequence_2'):
        prokkaFile = prokkaDir + s + '.gb'
        for i in list(annoDf.index):
            if (type(annoDf['ref_id_1'][i]) == 'float') and math.isnan(annoDf['ref_id_1'][i]):
                continue
            featureDict = getAnnotation(prokkaFile, annoDf['ref_id_1'][i], annoDf['ref_pos_1'][i], 
                          annoDf['ref_id_2'][i], annoDf['ref_pos_2'][i])
            print(featureDict)
            if 'coding' in featureDict:
                qual = featureDict['coding'].qualifiers
                if 'gene' in qual:
                    annoDf['coding'][i] = qual['gene']
                elif 'product' in qual:
                    annoDf['coding'][i] = qual['product']
            elif 'previous' in featureDict:
                prev = featureDict['previous']
                strand = str(prev.location).split('(')[1].replace(')', '')
                if strand == '-':
                    if 'gene' in prev.qualifiers:
                        annoDf['upstream'][i] = prev.qualifiers['gene']
                    elif 'product' in prev.qualifiers:
                        annoDf['upstream'][i] = prev.qualifiers['product']
                if strand == '+':
                    if 'gene' in prev.qualifiers:
                        annoDf['downstream'][i] = prev.qualifiers['gene']
                    elif 'product' in prev.qualifiers:
                        annoDf['downstream'][i] = prev.qualifiers['product']
annoDf


def exportBinaryMapping(df, outFileName):
    outFile = open(outFileName, 'w')
    for i in list(df.index):
        outFile.write('>' + i + '\n')
        outFile.write(df['sequence_1'][i] + '\n')
    outFile.close()

####IMPORT BLAST FEATURE RESULTS
#INPUT = inFileName: blast results
#dataframe with readids in rows, feature/5'/3' in columns, text description in data
#dictionary with feature/5'/3': readid: text
def importBlast(inFileName):
    features = {}
    getFeatures = False
    inFile = open(inFileName, 'r')
    for line in inFile:
        if 'Query= ' in line:
            query = line.strip().replace('Query= ', '')
            firstFeature = True
        if getFeatures:
            if line == '\n':
                getFeatures = False
            else:
                features[query] = line.strip()
                firstFeature = False
        if (' Features in this part of subject sequence:' in line) and (firstFeature):
            getFeatures = True
    inFile.close()
    print(features)

#NEXT: quantify coverage...

# Previous analysis

In [14]:
#IF a read maps to some but not others...
#WHICH of the unmapped reads are not shared by all?
#DO any of them cluster?

#readDict has subgroup (e.g. subA) : subDict
#subDict has mapped read ID : [[bwa filename, unmappedDict], [bwa filename, unmappedDict]]
#unmappedDict has readID : [list of read mapping info]
readDict = {}

#badDict has subgroup (e.g. subA) : badReads
#badReads has mapped read ID : [unmappedSet]
#unmappedSet is set of unmapped readIDs that appear in all samples (e.g. bad reads, not interesting)
badDict = {}

if not os.path.exists(readDir):
    os.makedirs(readDir)
for subdirs, dirs, files in os.walk(unmappedDir):
    for d in dirs:
        ####
        ####PREPARE DICTIONARY OF READS
        subDict = {}
        for f in os.listdir(unmappedDir + d):
            if 'contig' not in f:
                continue
            contig = f.split('_')[1]
            mapped = f.split('_')[3]            
            inFile = open(unmappedDir + d + '/' + f, 'r')
            unmappedDict = {}
            for line in inFile:
                line = line.strip().split('\t')
                readID = line[0]
                unmappedDict[readID] = line
            inFile.close()
            if mapped in subDict:
                subDict[mapped].append([f, unmappedDict])
            else:
                subDict[mapped] = [[f, unmappedDict]]
        readDict[d] = subDict
        ####
        ####IDENTIFY READS THAT DON'T MAP TO ANY SAMPLE
        badReads = {}
        for i, mapped in enumerate(subDict):
            unmappedSet = set()
            for j, result in enumerate(subDict[mapped]):
                f = result[0]
                unmappedDict = result[1]
                if j == 0:
                    unmappedSet = set(unmappedDict.keys())
                else:
                    unmappedSet = unmappedSet.intersection(unmappedDict.keys())
            #print(mapped + ': ' + str(len(unmappedSet)))
            badReads[mapped] = unmappedSet
        badDict[d] = badReads
        ####
        ####RECORD READS THAT MAP TO SOME BUT NOT OTHERS
        ####USE DIAMOND AGAINST NR?!?!
        readsOfInterest = {}
        for i, mapped in enumerate(subDict):
            for j, result in enumerate(subDict[mapped]):
                f = result[0]
                unmappedDict = result[1]
                for read in unmappedDict:
                    if read not in badReads[mapped]:
                        if read not in readsOfInterest:
                            seq = unmappedDict[read][9]
                            readsOfInterest[read] = seq
        ####
        ####EXPORT READS OF INTEREST FOR BLAST ANALYSIS
        if not os.path.exists(readDir):
            os.makedirs(readDir)
        outFileName = readDir + d + '_horizontal.fa'
        outFile = open(outFileName, 'w')
        for r in readsOfInterest:
            outFile.write('>' + r + '\n' + readsOfInterest[r] + '\n')
        outFile.close()
        #os.system('blastn -task megablast -db nt_blast/nt -query ' + outFileName + ' -out ' + \
        #          outFileName.replace('.fa', '.out'))
        #
        #
        #diamond blastx -d nr -q reads.fna -a reads -t <temporary directory>
        #('blastn megablast ')
        #os.system('diamond blastx -d nr_diamond/nr -q ' + outFileName + ' -a ' + \
        #          outFileName.replace('.fa', '') + ' -t tmp/')

Ran BLAST 2.6.1+ against nt on 5/15/2017  
Downloaded results in plain text for subgroupA and uploaded here

In [28]:
#featureDict has subgroup name : features
#features has query/read ID : features in this part of subject sequence
featureDict = {}

for subdirs, dirs, files in os.walk(unmappedDir):
    for d in dirs:
        ####
        ####IMPORT BLAST FEATURE RESULTS
        blastOutName = readDir + d + '_horizontal_Alignment.txt'
        features = {}
        blastOut = open(blastOutName, 'r')
        getFeatures = False
        for line in blastOut:
            if 'Query= ' in line:
                query = line.strip().replace('Query= ', '')
                firstFeature = True
            if getFeatures:
                if line == '\n':
                    getFeatures = False
                else:
                    features[query] = line.strip()
                    firstFeature = False
            if (' Features in this part of subject sequence:' in line) and (firstFeature):
                getFeatures = True
        blastOut.close()
        featureDict[d] = features
        ####
        ####MAKE DESCRIPTIVE DATAFRAME
        subDict = readDict[d]
        badReads = badDict[d]
        ctDict = {}
        for i, mapped in enumerate(subDict):
            for j, result in enumerate(subDict[mapped]):
                f = result[0]
                unmappedDict = result[1]
                contig = f.split('_')[1]
                if contig not in ctDict:
                    ctDict[contig] = {}
                for read in unmappedDict:
                    if read not in badReads[mapped]:
                        if read in features: #NOTE: not all reads had blast results...
                            currentFeature = features[read]
                            if currentFeature in ctDict[contig]:
                                ctDict[contig][currentFeature] += 1
                            else:
                                ctDict[contig][currentFeature] = 1
        df = pd.DataFrame.from_dict(ctDict)


,D17-102036,D17-102037,D17-102038,D17-102039,D17-102040,D17-102041,D17-102042,D17-102043,D17-102044,D17-102045,D17-102046,D17-102047,D17-102048,D17-102049,D17-102050,D17-102051,D17-102065
"1,4-alpha-glucan (glycogen) branching enzyme, GH-13-type",1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1-deoxy-D-xylulose 5-phosphate reductoisomerase,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
1-deoxy-D-xylulose-5-phosphate synthase,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
"2',3'-cyclic-nucleotide 2'-phosphodiesterase",1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2-acyl-glycerophospho-ethanolamine acyltransferase,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2-dehydro-3-deoxyphosphogluconate aldolase/4-hydroxy-2-ox...,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2-nitropropane dioxygenase,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0
2-octaprenylphenol hydroxylase,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
2-oxo-3-deoxygalactonate kinase,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
"2-oxo-hepta-3-ene-1,7-dioate hydratase",1.0,1.0,3.0,1.0,3.0,1.0,3.0,NaN,1.0,1.0,1.0,1.0,3.0,3.0,1.0,3.0,1.0


In [37]:
for i,row in df.iterrows():
    if row.sum() > 100:
        print(row)

D17-102036       1.0
D17-102037       1.0
D17-102038       1.0
D17-102039       1.0
D17-102040       1.0
D17-102041       1.0
D17-102042       1.0
D17-102043       1.0
D17-102044       1.0
D17-102045       1.0
D17-102046       NaN
D17-102047    2540.0
D17-102048       1.0
D17-102049    2540.0
D17-102050       1.0
D17-102051       1.0
D17-102065       1.0
Name: GntR family transcriptional regulator, dtype: float64
D17-102036     NaN
D17-102037     NaN
D17-102038    70.0
D17-102039     NaN
D17-102040    83.0
D17-102041     NaN
D17-102042    55.0
D17-102043     NaN
D17-102044     NaN
D17-102045     NaN
D17-102046     NaN
D17-102047     NaN
D17-102048    55.0
D17-102049    83.0
D17-102050     NaN
D17-102051    83.0
D17-102065     NaN
Name: copper resistance protein CopD, dtype: float64
D17-102036       1.0
D17-102037       1.0
D17-102038       1.0
D17-102039       1.0
D17-102040       1.0
D17-102041       1.0
D17-102042       NaN
D17-102043       1.0
D17-102044       1.0
D17-102045       1